In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_columns', 30)

In [ ]:
data_class = pd.read_csv('../data/weatherAUS.csv')
data_class

In [ ]:
data_reg = pd.read_csv()
data_reg